In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cude':
    torch.cuda.manual_seed_all(777)
print(device)

cpu


In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

C:\Users\Admin\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [6]:
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        # L1 Imglin shape = (?, 28, 28, 1)
        #    Conv       ->  (?, 28, 28, 32)
        #    Pool       ->  (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        #
        #
        #
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)      # Flatten them for FC
        out = self.fc(out)
        return out

In [7]:
model = CNN().to(device)

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        #
        #
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    
print('Learning Finished!')

Learning started. It takes sometime.
[Epoch :    1] cost = 0.215919703
[Epoch :    2] cost = 0.0592471883
[Epoch :    3] cost = 0.0439879335
[Epoch :    4] cost = 0.03571238
[Epoch :    5] cost = 0.0294649471
[Epoch :    6] cost = 0.0244152118
[Epoch :    7] cost = 0.0209837463
[Epoch :    8] cost = 0.0179216843
[Epoch :    9] cost = 0.0148106636
[Epoch :   10] cost = 0.01178966
[Epoch :   11] cost = 0.0114992568
[Epoch :   12] cost = 0.0089272093
[Epoch :   13] cost = 0.00718684634
[Epoch :   14] cost = 0.00687589264
[Epoch :   15] cost = 0.00794217829
Learning Finished!


In [12]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.98580002784729
